# Why You want Weighted Training for Jane Street
**Intro**

When the accuracy of the model is not the final objective but a solution to a real world problem is, weighted training is often a valuable tool.

In many notebooks i have seen the score correlated to Wegiht* Resp, and calculation of that then used as a Metric. 
But i see few use it in the training process. Weighted training is extremly usefull in particular situations, where a misclassification can be particularrly damaging ( a high loss in this case ) or particularly beneficial ( high profit). 
Since Jane street fits into this application and it was often not used i set out to see it's  effectivness in this competition.

I use the LGBMClassifier from lightgbm witch supports weight training.
I have not applied any feature engineering at all nor parametrization of the lgmb as the score it achieves is not important. The increment in score achievable with this technic is the focus.
It gets shown to be a must have in the pipeline.
i create a fertile ground for more work.









I am sure the people high up on the leader board are using this is some form,but it is often overlooked or obscure, and this is one of it's perfect applicationd !
See how many of you it benefit !  if you like it please Upvote ;) . 

I use and thank [@Luis Moneda](https://www.kaggle.com/lgmoneda)for some snabbits of code from ( maybe check it out !) [this notebook](https://www.kaggle.com/lgmoneda/jane-temporal-feature-selection-with-shap/comments) for the evaluation metrics,data split, graph (adapted) and submission code.

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import math
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import janestreet
import warnings

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

import warnings

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (12, 4)
warnings.filterwarnings('ignore')

# And for those who don't know what weighted training is ..

Its theoretical way of working is fairly simply. To Each Label will be attributed a weight, and this weight will be used  as a way of sizing the different errors of misclassification while computing the loss function.
It such a way it can call the attention more onto what matters in a scenario. When the precision of the model is not the main objective but a real world problem is.
Since we are trying to make money, the important $$ trades should matter more. Weighted training makes the models we create see what matters. 
But how best to do that ?


# Metrics
This is the formula witch i see going around. 
( DISCALIMER - i have not seen or checked if this reflects the actual scoring sistem created by the Competition creators. it is just the most widespread method) 
pi= ∑j(weightij∗respij∗actionij)

t= ∑pi/∑p**2i * √250|i|

(i being the number of unique dates in test set ).

utitily = min(max(t,0),6)∑pi.



In [ ]:
def utility_score_numba(date, weight, resp, action):
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / len(Pi))
    u = min(max(t, 0), 6) * np.sum(Pi)
    return u

def jane_utility(data, action_column="action"):
    return utility_score_numba(data["date"].values, 
                               data["weight"].values, 
                               data["resp"].values, 
                               data[action_column].values)

Train and Test Split + keeping back some of the last days of data ( 100) labled -out of time
- just as the competition data will be from an unseen future. 

In [ ]:
train_data = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")

TIME_SPLIT= 400
in_time= train_data.loc[train_data['date']<TIME_SPLIT].copy()
out_of_time= train_data.loc[train_data['date']>=TIME_SPLIT].copy()

train, test = train_test_split(in_time, 
                           test_size=0.2, 
                           random_state=42)



In [ ]:
#clean up 
import gc 
del train_data
gc.collect()

In [ ]:
#Some minor settings
TARGET='action'
features = [f'feature_{x}' for x in range(130)] 

#Setting my will do do only Finiancially positive trades
train['action']= train['resp']>0
test['action']= test['resp']>0
out_of_time['action']= out_of_time['resp']>0


# Workshop,different simple methods of calculating weights for the labels.

working with different distributions of weights, 
this is the playground.

In [ ]:
#Different simple methods of calculating weights for the labels
#Create a column for each different distribution
train['none']= np.ones(train.shape[0],int)
train['weight']=abs(train['weight'])
train['mix']= abs(train['resp'])*train['weight']
train['res']= abs(train['resp'])

# " And if that doesn't work, try the root or the log"
#- Andrew Ng.
train['mix1']= abs(train['resp'])*(train['weight'].transform('sqrt'))
train['mix2']= abs(train['resp'])*(train['weight'].transform('log').clip(lower=0.01, upper=None)*train['weight'].transform('sqrt'))
train['mix3']= abs(train['resp'])*(train['weight'].transform('sqrt').transform('sqrt'))


In [ ]:
print('Baseline')
for weight_method in ['none','weight','mix','res','mix1','mix2','mix3']:
    print('\nTraining now with {0} as method for calculating training weights'.format(weight_method))
    model = LGBMClassifier()
    model.fit(train[features], train[TARGET],sample_weight= train[weight_method])


    test["challenger"] = model.predict_proba(test[features])[:, 1]
    out_of_time["challenger"] = model.predict_proba(out_of_time[features])[:, 1]

    test["{}_action".format(weight_method)] = model.predict(test[features])
    out_of_time["{}_action".format(weight_method)] = model.predict(out_of_time[features])

    print("Test AUC (in time): {:.6f}".format(roc_auc_score(test[TARGET], test["challenger"])))
    print("Out of time AUC: {:.6f}".format(roc_auc_score(out_of_time[TARGET], out_of_time["challenger"])))
    print("Test Jane Utility (in time): {:.2f}".format(jane_utility(test, "{}_action".format(weight_method))))
    print("Out of time Jane Utility: {:.2f}".format(jane_utility(out_of_time, "{}_action".format(weight_method))))
    print("-----------")
    
    

**What this shows**:
to start off the main advantage of weighted training is showing the main gool (utility score) is improving when the accuracy is not. ( even doing slightly worse) 

If we pay more attention to the Big Trades, weighted by - Weight we miss out on a lot of opportunities. Score pretty low in comparison to the bench mark. It is not a reasonable practice. 

If we give more attention to the highly $$ profitable trades, weighted by Weight*resp , 
on the witheld Test data we make significantly more Dough ! 
It seems not to generalize well into the more distant future - days we haven't seen (out_of[training]_time ) 

Attention to being profitable with Resp seems to be a good trade off !! All round increase. 

But things really get exicting with Mix3 !

a 20% increase on the future for free just there


In [ ]:
pd.concat([test, out_of_time]).groupby("date").apply(lambda x: np.sum(x["resp"] * x["weight"] * x["none_action"])).rolling(60).mean().plot(label="No weights (benchmark)", color="purple")
# pd.concat([test, out_of_time]).groupby("date").apply(lambda x: np.sum(x["resp"] * x["weight"] * x["weight_action"])).rolling(60).mean().plot(label="weight")
# pd.concat([test, out_of_time]).groupby("date").apply(lambda x: np.sum(x["resp"] * x["weight"] * x["mix_action"])).rolling(60).mean().plot(label="Weight*resp ", color="green")
pd.concat([test, out_of_time]).groupby("date").apply(lambda x: np.sum(x["resp"] * x["weight"] * x["res_action"])).rolling(60).mean().plot(label="resp")
pd.concat([test, out_of_time]).groupby("date").apply(lambda x: np.sum(x["resp"] * x["weight"] * x["mix3_action"])).rolling(60).mean().plot(label="mix3")
xmin, xmax, ymin, ymax = plt.axis()
plt.vlines(TIME_SPLIT, ymin, ymax, linestyle="dotted", color="red", label="Out of time split")
plt.legend(bbox_to_anchor=(1.05, 1.0))
plt.title("Performance moving average of 60 periods window for both test and out of time periods", pad=16)
plt.ylabel("sum(Weight * Resp * Action)")
plt.xlabel("Date")
plt.show()

As the graphs shows some days it pays to be more focused on on thing a some days on another. Trends that some Moveing Average and some feature engineering may undercover and help take advantage out of.

Here simply focusing the attention on certain trades with a more noticeable resp made for more constant and higher gains than the model trained normaly. 

As you can see from the graph it seems a bit more constant & stable. **Slow and steady wins the race**?  
Not so sure, sometimes going big helps ! 
look at Mix3 ! up 20+% !
- to be continued and developed



In [ ]:
#clean up before restarting
import gc 
del out_of_time,in_time ,test,train,model
gc.collect()

** 
# On the Lb 
The best mothod for weight training appears to be the Resp, ( +170 above the baseline for this minimalistic lgbm) 
So the validation set i chose (days after 400) may have been overfitted with Mix3. 

I set out to do some cross validation and find a more realistic( akin to the Lb) validation set.
Any knowledge about how many days that contains ?  

Any tips,ideas, upvotes and comments are greatly appreciated 


# **UPDATE**

I aimed to make my train and validation set more reflect the Competition Data distribution beetween Training data and the private leader board Test set.
Giving the High score on the board and the lentgh of time it took to score i thought we were only given a small part of the data and we were being tested many days into the future. 
( this was supported by what i read [here](https://www.kaggle.com/c/jane-street-market-prediction/discussion/200495) about the time period. 

So below i re-scored all the algorithms to reflect what this might change. The validation split is now done at day 100( the realationship beetween time seen at time tested over is key). Here we can see how poorly some weighted trainging made us do in the future. ( probably due to some overfitting of outsiders) 
This new division shows us a more realistic view into the benefits of wighted training in the competition and also reflects why using just resp scores better than other formulas on the LeaderBoard.


In [ ]:
train_data = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")

TIME_SPLIT= 100
in_time= train_data.loc[train_data['date']<TIME_SPLIT].copy()
out_of_time= train_data.loc[train_data['date']>=TIME_SPLIT].copy()

train, test = train_test_split(in_time, 
                           test_size=0.2, 
                           random_state=42)

#Some minor settings
TARGET='action'
features = [f'feature_{x}' for x in range(130)] 

#Setting my will do do only Finiancially positive trades
train['action']= train['resp']>0
test['action']= test['resp']>0
out_of_time['action']= out_of_time['resp']>0

#Different simple methods of calculating weights for the labels
#Create a column for each different distribution
train['none']= np.ones(train.shape[0],int)
train['weight']=abs(train['weight'])
train['mix']= abs(train['resp'])*train['weight']
train['res']= abs(train['resp'])

# " And if that doesn't work, try the root or the log"
#- Andrew Ng.
train['mix1']= abs(train['resp'])*(train['weight'].transform('sqrt'))
train['mix2']= abs(train['resp'])*(train['weight'].transform('log').clip(lower=0.01, upper=None)*train['weight'].transform('sqrt'))
train['mix3']= abs(train['resp'])*(train['weight'].transform('sqrt').transform('sqrt'))


print('Baseline')
for weight_method in ['none','weight','mix','res','mix1','mix2','mix3']:
    print('\nTraining now with {0} as method for calculating training weights'.format(weight_method))
    model = LGBMClassifier()
    model.fit(train[features], train[TARGET],sample_weight= train[weight_method])


    test["challenger"] = model.predict_proba(test[features])[:, 1]
    out_of_time["challenger"] = model.predict_proba(out_of_time[features])[:, 1]

    test["{}_action".format(weight_method)] = model.predict(test[features])
    out_of_time["{}_action".format(weight_method)] = model.predict(out_of_time[features])

    print("Test AUC (in time): {:.6f}".format(roc_auc_score(test[TARGET], test["challenger"])))
    print("Out of time AUC: {:.6f}".format(roc_auc_score(out_of_time[TARGET], out_of_time["challenger"])))
    print("Test Jane Utility (in time): {:.2f}".format(jane_utility(test, "{}_action".format(weight_method))))
    print("Out of time Jane Utility: {:.2f}".format(jane_utility(out_of_time, "{}_action".format(weight_method))))
    print("-----------")
    

Here, Mix 3 and res were still good but here, with this realistic distribution of ( seen and not seen time )
Mix1 emerged as a Winner. i am happy to say this validation method was backed up by the Lb.
and the simple lgbm now scores with weight training with Mix1 4294. Vs the baseline of 3807-
a 13% percent increase in score ont the baseline. 
Since in this book we had a + 40 % increase above the baseline, while going in right direction with the more unfair split in train and test data i still do not think i am close to having a really functional valdiation set. 

Mathematically i am not supprised Mix 1 won. It was a good runner all along and mathematically it also makes sense, the realtionship between resp and weight seemed to be inservly proportinal in an exponential way as the grafic below shows.
Since resp x Weight is in the evaulation metric it made sense to have both in the weight distribution, eliminating the esponential likeness int the relationship avoids in the training giving a disproportionate weight to a few points. Flatening it out in training. 

In [ ]:
#mix
import seaborn as sns
train_data['t_weight']=abs(train_data['resp'])*train_data['weight']
sns.scatterplot(x=train_data['resp'],y=train_data['weight'],hue=train_data['t_weight'])
plt.title("Distribution of training weights calc. using : weight* resp  (aka Mix) ", pad=16)

In [ ]:
#Mix1
import seaborn as sns
train_data['t_weight']=abs(train_data['resp'])*train_data['weight'].transform('sqrt')
sns.scatterplot(x=train_data['resp'],y=train_data['weight'],hue=train_data['t_weight'])
plt.title("Distribution of training weights calc. using : sqrt(weight)* resp  ( aka Mix1)", pad=16)

Here the important area at a 45 degree angle is much better distributed than above ! 
the difference is in the formula beetween the winning mix1 below and resp x Weight
below the how well the  models faired in the time line, when trained with the new train and validation set.

In [ ]:
pd.concat([test, out_of_time]).groupby("date").apply(lambda x: np.sum(x["resp"] * x["weight"] * x["none_action"])).rolling(60).mean().plot(label="No weights (benchmark)", color="purple")
# pd.concat([test, out_of_time]).groupby("date").apply(lambda x: np.sum(x["resp"] * x["weight"] * x["weight_action"])).rolling(60).mean().plot(label="weight")
pd.concat([test, out_of_time]).groupby("date").apply(lambda x: np.sum(x["resp"] * x["weight"] * x["mix3_action"])).rolling(60).mean().plot(label="mix3 ", color="green")
pd.concat([test, out_of_time]).groupby("date").apply(lambda x: np.sum(x["resp"] * x["weight"] * x["res_action"])).rolling(60).mean().plot(label="resp")
pd.concat([test, out_of_time]).groupby("date").apply(lambda x: np.sum(x["resp"] * x["weight"] * x["mix1_action"])).rolling(60).mean().plot(label="mix1")
xmin, xmax, ymin, ymax = plt.axis()
plt.vlines(TIME_SPLIT, ymin, ymax, linestyle="dotted", color="red", label="Out of time split")
plt.legend(bbox_to_anchor=(1.05, 1.0))
plt.title("Performance moving average of 60 periods window for both test and out of time periods", pad=16)
plt.ylabel("sum(Weight * Resp * Action)")
plt.xlabel("Date")
plt.show()

Very different from above right ? that is beacuase of the time lapse, models traidneed in the past rarely reflect with accuracy- or have problems in doing so- the current state. This downward trend is Why Luis Moneda did his goodnotebook i linked. - and why normally you would want to use an up to date model for stock trading.
Note when things go badly as in figure - day 450 ish it pays to have just paid attention to resp.
too bad we have no real time resp information in the competition.

# So in practice, import the data  + 4 lines of code 
and submit and score + 4000

In [ ]:
#clean up before restarting
import gc 
del out_of_time,in_time,test,train,model,train_data
gc.collect()

In [ ]:
#Download all the data
data = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")

#-5 lines of code
#create Weights
data['mix1']= abs(data['resp'])*data['weight'].transform('sqrt')
#set Features, target and classifications
data['TARGET']= data['resp']>0
features = [f'feature_{x}' for x in range(130)] 
#train Model
model = LGBMClassifier()
model.fit(data[features], data['TARGET'],sample_weight= data['mix1'])


#submission
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df["action"] = model.predict(test_df[features]).astype(int)
    env.predict(sample_prediction_df)

Take away: 
i hope i have convinced you of the use for Weighted Training, and as indicated above
( abs(resp) x sqrt(Weight) ) seems the way to go as a weight distribution. (aka Mix1)

Enjoy your day !